In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf

Wave equation:
$$
\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}
$$

Finite difference method:
$$
\frac{\partial^2 a}{\partial z^2} = \frac{a_{i+1} - 2 a_i + a_{i-1}}{(\Delta z)^2}
$$

FDM (explicit) applied to wave equation:

$$
\frac{u^{i+1}_j - 2 u^i_j + u^{i-1}_j}{(\Delta t)^2} = c^2 \frac{u^i_{j+1} - 2 u^i_j + u^i_{j-1}}{(\Delta x)^2} \\
u^{i+1}_j = 2 u^i_j - u^{i-1}_j + \left(\frac{c \Delta t}{\Delta x}\right)^2 ( u^i_{j+1} - 2 u^i_j + u^i_{j-1} )
$$

Add damping:
$$
\frac{\partial^2 u}{\partial t^2} + \lambda c \frac{\partial u}{\partial t} = c^2 \frac{\partial^2 u}{\partial x^2} \\
\frac{u^{i+1}_j - 2 u^i_j + u^{i-1}_j}{(\Delta t)^2} + \lambda c \frac{u^i_j - u^{i-1}_j}{\Delta t} = c^2 \frac{u^i_{j+1} - 2 u^i_j + u^i_{j-1}}{(\Delta x)^2} \\
u^{i+1}_j = (2 - \lambda c \Delta t)\, u^i_j - (1 - \lambda c \Delta t)\, u^{i-1}_j + \left(\frac{c \Delta t}{\Delta x}\right)^2 ( u^i_{j+1} - 2 u^i_j + u^i_{j-1} )
$$


In [ ]:
delta_t = 44000**-1  # s
delta_x = .01  # m
# wave_speed = wave_length * freq
wave_speed = 220  # ms^-1
string_length = 0.5  # m
decay = 0.05

In [ ]:
xs = delta_x * np.arange(int(string_length / delta_x))

# start = np.sin(np.pi * xs / np.max(xs))
loc = .1 ; start = np.where(xs < loc, (xs / loc), (np.max(xs) - xs) / (np.max(xs) - loc))  # triangle
# start = (xs == xs[len(xs) // 2]).astype(np.float)  # delta function

displacements = [start, start]
for _ in range(int(delta_t ** -1)):
    d_1 = displacements[-2]
    d = displacements[-1]
    displacements.append(np.pad((2 - decay * wave_speed * delta_t) * d[1:-1]
                                - (1 - decay * wave_speed * delta_t) * d_1[1:-1]
                                + (wave_speed * delta_t / delta_x)**2 * (d[2:] - 2 * d[1:-1] + d[:-2]), 1))

plt.figure(figsize=(12, 4))
plt.plot(xs, np.zeros_like(xs), color='b')
plt.plot(xs, start, color='r')
for d in displacements[2::100][:100]:
    plt.plot(xs, d, color='k', alpha=.1)

d0 = np.stack(displacements)[:, 1]
ts = delta_t * np.arange(len(d0))
plt.figure(figsize=(12, 4))
plt.plot(ts, d0)
plt.figure(figsize=(12, 4))
plt.plot(ts, d0)
plt.xlim((0, .05))

In [ ]:
# !pip install soundfile
sf.write('tri.1.wav', d0, int(delta_t**-1))

In [ ]:
from IPython.display import display, Audio, HTML
for f in ['sine.wav', 'tri.1.wav', 'tri.2.wav', 'delta.wav']:
    display(HTML(f'<b>{f}</b>'))
    display(Audio(f))
# display(Audio('tri.2.wav'))